In [1]:
!pip install optuna

     |████████████████████████████████| 308 kB 8.5 MB/s 
     |████████████████████████████████| 80 kB 7.0 MB/s 
     |████████████████████████████████| 210 kB 60.7 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |████████████████████████████████| 149 kB 61.1 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 
     |████████████████████████████████| 113 kB 52.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=1e040e2b655e3debf2424e9225c8f5220ef8163bf1f1bf7221b61cfcaaa574c0
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
import logging
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import LBFGS

from tqdm import tqdm
from IPython.display import clear_output

import optuna
from optuna.trial import TrialState

torch.manual_seed(42)
np.random.seed(42)

In [3]:
def clear_env():
  !rm -fv *.py
  !rm -fv *.py*
  !rm -fv *.pl
  !rm -rfv H2-H2O/
  !rm -rfv src/

In [4]:
clear_env()

In [5]:
!mkdir -v src/

!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/Makefile
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/molecule_simple.hh
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/monomial.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/monomial.hh
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/msa.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/poly_basis.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/polynomial.cpp
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/polynomial.hh
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/src/utility.hh

!mv -v Makefile src/
!mv -v molecule_simple.hh src/
!mv -v monomial.cpp src/
!mv -v monomial.hh src/
!mv -v msa.cpp src/
!mv -v poly_basis.cpp src/
!mv -v polynomial.cpp src/
!mv -v polynomial.hh src/
!mv -v utility.hh src/

clear_output()

In [6]:
from zipfile import ZipFile

!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/H2-H2O/points.zip

!mkdir -v H2-H2O

with ZipFile('points.zip', 'r') as f:
  f.extractall()

!mv -v points.dat H2-H2O/
!rm -vf points.zip

--2022-02-13 01:00:35--  https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/H2-H2O/points.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2794987 (2.7M) [application/zip]
Saving to: ‘points.zip’

points.zip          100%[===================>]   2.67M  --.-KB/s    in 0.05s   

2022-02-13 01:00:35 (51.5 MB/s) - ‘points.zip’ saved [2794987/2794987]

mkdir: created directory 'H2-H2O'
renamed 'points.dat' -> 'H2-H2O/points.dat'
removed 'points.zip'


In [7]:
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/genpip.py
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/postmsa.pl

from genpip import build_lib
from genpip import run_msa
from genpip import compile_dlib

logging.info("building MSA library...\n")
build_lib()

order = "3"
symmetry = "2 2 1"
wdir = "H2-H2O"
config_fname = "points.dat"

run_msa(order, symmetry, wdir)

!perl postmsa.pl H2-H2O 3 2 2 1
!gfortran -fPIC -shared H2-H2O/basis_2_2_1_3.f90 -o H2-H2O/basis_2_2_1_3.so

--2022-02-13 01:00:36--  https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/genpip.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3162 (3.1K) [text/plain]
Saving to: ‘genpip.py’

genpip.py           100%[===================>]   3.09K  --.-KB/s    in 0s      

2022-02-13 01:00:36 (33.3 MB/s) - ‘genpip.py’ saved [3162/3162]

--2022-02-13 01:00:36--  https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/postmsa.pl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6936 (6.8K) [text/plain]
Saving to: ‘postmsa.p

In [8]:
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/dataset.py

from dataset import PolyDataset

clear_output()

In [9]:
HTOCM = 2.194746313702e5

# scaling of X (polynomial values) and y (energies)
# TODO: check out several other scaling transformations (minmax, etc)
SCALE_OPTIONS = [None, "std"]
SCALE_PARAMS = {
    "Xscale" : "std",
    "yscale" : "std"
}

class StandardScaler:
    def fit(self, x):
        self.mean = x.mean(0, keepdim=True)
        self.std = x.std(0, unbiased=False, keepdim=True)

    def transform(self, x):
        c = torch.clone(x)
        c -= self.mean
        c /= self.std
        return torch.nan_to_num(c, nan=1.0)

class IdentityScaler:
    def fit(self, x):
        pass

    def transform(self, x):
        return x

In [10]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [12]:
class EarlyStopping:
    def __init__(self, patience=10, tol=0.1, chk_path='checkpoint.pt'):
        """
        patience : how many epochs to wait after the last time the monitored quantity [validation loss] has improved
        tol:       minimum change in the monitored quantity to qualify as an improvement
        path:      path for the checkpoint to be saved to
        """
        self.patience = patience
        self.tol = tol
        self.chk_path = chk_path

        self.counter = 0
        self.best_score = None
        self.status = False

    def __call__(self, score, model):
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)

        elif score < self.best_score and (self.best_score - score) > self.tol:
            self.best_score = score
            self.counter = 0
            self.save_checkpoint(model)

        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.status = True

        logging.debug("Best validation RMSE: {:.2f}; current validation RMSE: {:.2f}".format(self.best_score, score))
        logging.debug("ES counter: {}; ES patience: {}".format(self.counter, self.patience))

    def save_checkpoint(self, model):
        logging.debug("Saving the checkpoint")
        torch.save(model.state_dict(), self.chk_path)

In [13]:
def split_train_val_test(X, y):
    """
    # TODO: implement energy-based splitting of dataset
    """
    sz = y.size()[0]

    ids = np.random.permutation(sz)
    val_start  = int(sz * 0.8)
    test_start = int(sz * 0.9)
    train_ids = ids[:val_start]
    val_ids = ids[val_start:test_start]
    test_ids = ids[test_start:]

    X_train, y_train = X[train_ids], y[train_ids]
    X_val, y_val     = X[val_ids], y[val_ids]
    X_test, y_test   = X[test_ids],  y[test_ids]

    assert SCALE_PARAMS["Xscale"] in SCALE_OPTIONS
    if SCALE_PARAMS["Xscale"] == "std":
        xscaler = StandardScaler()
    elif SCALE_PARAMS["Xscale"] == None:
        xscaler = IdentityScaler()
    else:
        raise ValueError("unreachable")

    assert SCALE_PARAMS["yscale"] in SCALE_OPTIONS
    if SCALE_PARAMS["yscale"] == "std":
        yscaler = StandardScaler()
    elif SCALE_PARAMS["yscale"] == None:
        yscaler = IdentityScaler()
    else:
        raise ValueError("unreachable")

    xscaler.fit(X_train)
    Xtr       = xscaler.transform(X)
    Xtr_train = xscaler.transform(X_train)
    Xtr_val   = xscaler.transform(X_val)
    Xtr_test  = xscaler.transform(X_test)

    yscaler.fit(y_train)
    ytr       = yscaler.transform(y)
    ytr_train = yscaler.transform(y_train)
    ytr_val   = yscaler.transform(y_val)
    ytr_test  = yscaler.transform(y_test)

    return Xtr_train, ytr_train, Xtr_val, ytr_val, Xtr_test, ytr_test, xscaler, yscaler

In [22]:
def define_model(trial, NPOLY):
    # TODO: maybe add a little Dropout as a means to counteract overfitting

    # we optimize the number of layers and hidden units
    n_layers = trial.suggest_int("n_layers", low=1, high=5)

    layers = []

    in_features = NPOLY
    for i in range(n_layers):
        out_features = trial.suggest_int("n_hidden_l{}".format(i), low=3, high=40)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.Tanh())

        in_features = out_features

    layers.append(nn.Linear(in_features, 1))

    return nn.Sequential(*layers)

In [20]:
def build_model(trial):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    d = torch.load("H2-H2O/dataset.pt")
    X, y = d["X"], d["y"]

    X_train, y_train, X_val, y_val, X_test, y_test, xscaler, yscaler = split_train_val_test(X, y)
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_val, y_val = X_val.to(device), y_val.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)
    X, y = X.to(device), y.to(device)

    if SCALE_PARAMS["yscale"] == "std":
        rmse_descaler = torch.linalg.norm(yscaler.std)
    elif SCALE_PARAMS["yscale"] == None:
        rmse_descaler = 1.0
    else:
        raise ValueError("unreachable")

    NPOLY = X_train.size()[1]
    model = define_model(trial, NPOLY)
    model.double()
    model.to(device)

    optimizer = LBFGS(model.parameters(), lr=1.0, line_search_fn='strong_wolfe', tolerance_grad=1e-14, tolerance_change=1e-14, max_iter=100)

    METRIC_TYPES = ['RMSE', 'MSE']
    METRIC_TYPE = 'MSE'
    assert METRIC_TYPE in METRIC_TYPES

    logging.info("METRIC_TYPE = {}".format(METRIC_TYPE))

    if METRIC_TYPE == 'MSE':
        metric = nn.MSELoss()
    elif METRIC_TYPE == 'RMSE':
        metric = RMSELoss()
    else:
        raise ValueError("unreachable")

    SCHEDULER_PATIENCE = 5
    RMSE_TOL  = 0.1 # cm-1
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, threshold=RMSE_TOL, threshold_mode='abs', cooldown=2, patience=SCHEDULER_PATIENCE)

    ES_START_EPOCH = 10
    chk_path = 'checkpoint_{}.pt'.format(trial.number)
    es = EarlyStopping(patience=10, tol=RMSE_TOL, chk_path=chk_path)

    MAX_EPOCHS = 500

    ################ START TRAINING #######################
    for epoch in range(MAX_EPOCHS):
        def closure():
            optimizer.zero_grad()
            y_pred = model(X_train)
            loss = metric(y_pred, y_train)
            loss.backward()
            return loss

        optimizer.step(closure)
        loss = closure()

        with torch.no_grad():
            pred_val = model(X_val)
            loss_val = metric(pred_val, y_val)

        if METRIC_TYPE == 'MSE':
            rmse_val   = torch.sqrt(loss_val) * rmse_descaler * HTOCM
            rmse_train = torch.sqrt(loss)     * rmse_descaler * HTOCM
        elif METRIC_TYPE == 'RMSE':
            rmse_val   = loss_val * rmse_descaler * HTOCM
            rmse_train = loss     * rmse_descaler * HTOCM
        else:
            raise ValueError("unreachable")

        scheduler.step(rmse_val)
        lr = get_lr(optimizer)
        logging.info("Current learning rate: {:.2e}".format(lr))

        if epoch > ES_START_EPOCH:
            es(rmse_val, model)

            if es.status:
                logging.info("Invoking early stop")
                break

        logging.info("Epoch: {}; train RMSE: {:.2f} cm-1; validation RMSE: {:.2f}\n".format(
            epoch, rmse_train, rmse_val
        ))
    ################ END TRAINING #######################

    model_params = torch.load(chk_path)
    model.load_state_dict(model_params)
    logging.info("\nReloading best model from the last checkpoint...")

    with torch.no_grad():
        pred_val = model(X_val)
        loss_val = metric(pred_val, y_val)

        pred_test = model(X_test)
        loss_test = metric(pred_test, y_test)

        pred_full = model(X)
        loss_full = metric(pred_full, y)

        if METRIC_TYPE == 'MSE':
            rmse_val  = torch.sqrt(loss_val) * rmse_descaler * HTOCM
            rmse_test = torch.sqrt(loss_test) * rmse_descaler * HTOCM
            rmse_full = torch.sqrt(loss_full) * rmse_descaler * HTOCM
        elif METRIC_TYPE == 'RMSE':
            rmse_val  = loss_val * rmse_descaler * HTOCM
            rmse_test = loss_test * rmse_descaler * HTOCM
            rmse_full = loss_full * rmse_descaler * HTOCM

        logging.info("Final evaluation:")
        logging.info("Validation RMSE: {:.2f} cm-1".format(rmse_val))
        logging.info("Test RMSE:       {:.2f} cm-1".format(rmse_test))
        logging.info("Full RMSE:       {:.2f} cm-1".format(rmse_full))

    return rmse_val

In [16]:
# `force = True` -- removes any other active handlers
logger = logging.getLogger('my_logger')
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.DEBUG, force=True)

wdir     = './H2-H2O'
order    = "3"
symmetry = "2 2 1"
dataset = PolyDataset(wdir=wdir, config_fname="points.dat", order=order, symmetry=symmetry)

X, y = dataset.X, dataset.y
torch.save({"X" : X, "y" : y}, "H2-H2O/dataset.pt")

INFO:root:working directory: ./H2-H2O
[INFO] working directory: ./H2-H2O
INFO:root:configuration file: ./H2-H2O/points.dat
[INFO] configuration file: ./H2-H2O/points.dat
INFO:root:loading configurations...
[INFO] loading configurations...
INFO:root:detected NATOMS = 5
[INFO] detected NATOMS = 5
INFO:root:detected NCONFIGS = 44623
[INFO] detected NCONFIGS = 44623
INFO:root:preparing the coordinates..
[INFO] preparing the coordinates..
INFO:root:Done.
[INFO] Done.
INFO:root:detected NMON  = 39
[INFO] detected NMON  = 39
INFO:root:detected NPOLY = 102
[INFO] detected NPOLY = 102
INFO:root:Preparing the polynomials...
[INFO] Preparing the polynomials...
INFO:root:Done.
[INFO] Done.


In [25]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="minimize")
study.optimize(build_model, n_trials=100, timeout=3600)

pruned_trials   = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

logging.info("Study statistics:")
logging.info("  Number of finished trials: {}".format(study.trials))
logging.info("  Number of pruned trials:   {}".format(pruned_trials))
logging.info("  Number of complete trials: {}".format(complete_trials))

best_trial = study.best_trial
logging.info("Best trial:")
logging.info("  Best target value: {}".format(best_trial.value))
logging.info("  Parameters:")

for key, value in best_trial.params.items():
  logging.info("    {}: {}".format(key, value))

Streaming output truncated to the last 5000 lines.
[INFO] Current learning rate: 1.00e+00
INFO:root:Epoch: 6; train RMSE: 27.35 cm-1; validation RMSE: 36.03

[INFO] Epoch: 6; train RMSE: 27.35 cm-1; validation RMSE: 36.03

INFO:root:Current learning rate: 1.00e+00
[INFO] Current learning rate: 1.00e+00
INFO:root:Epoch: 7; train RMSE: 25.66 cm-1; validation RMSE: 35.71

[INFO] Epoch: 7; train RMSE: 25.66 cm-1; validation RMSE: 35.71

INFO:root:Current learning rate: 1.00e+00
[INFO] Current learning rate: 1.00e+00
INFO:root:Epoch: 8; train RMSE: 24.32 cm-1; validation RMSE: 38.31

[INFO] Epoch: 8; train RMSE: 24.32 cm-1; validation RMSE: 38.31

INFO:root:Current learning rate: 1.00e+00
[INFO] Current learning rate: 1.00e+00
INFO:root:Epoch: 9; train RMSE: 23.30 cm-1; validation RMSE: 40.38

[INFO] Epoch: 9; train RMSE: 23.30 cm-1; validation RMSE: 40.38

INFO:root:Current learning rate: 1.00e+00
[INFO] Current learning rate: 1.00e+00
INFO:root:Epoch: 10; train RMSE: 22.28 cm-1; validatio